## Creating embeddings on GPU
This is usually a bit faster, but it seams my gpu has to little memory for the model and the tokens at the same time.
So this does not really work

In [1]:
import wget
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import time
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertModel

C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\AppData\Local\Continuum

In [2]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\"
docs = pd.read_csv(path + "crod_19_only_rel.csv")
docs.shape

C:\Users\User\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(37279, 21)

In [3]:
def tokenize(model, tokenizer, sentence):
    
    # Tokens comes from a process that splits the input into sub-entities with interesting linguistic properties.
    tokens = tokenizer.tokenize(sentence.lower())

    # This is not sufficient for the model, as it requires integers as input, 
    # not a problem, let's convert tokens to ids.
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    #print(tokens_ids)

    # Add the required special tokens
    tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)
    #print(tokens_ids)
    
    # We need to convert to a Deep Learning framework specific format, let's use PyTorch for now.
    tokens_pt = torch.tensor([tokens_ids])

    # The length of the tokens can not be more than 512
    if len(tokens_pt[0]) > 512:
        tokens_pt = torch.tensor([tokens_pt[0][0:512].tolist()])
    
    tokens_pt = tokens_pt.cuda()
        
    # Now we're ready to go through BERT with out input
    outputs, pooled = model(tokens_pt)
    return pooled

In [4]:
model_names = ['bert-base-uncased', "allenai/scibert_scivocab_uncased", "gsarti/covidbert-nli"]

In [5]:
model_id = 0

In [6]:
bert_tokenizer = 0
bert_model = 0

model_id = 0

if model_id == 0:
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased').cuda()
if model_id == 1:
    scibert_tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
    scibert_model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased").cuda()
if model_id == 2:
    covid_tokenizer = AutoTokenizer.from_pretrained("gsarti/covidbert-nli")
    covid_model = AutoModel.from_pretrained("gsarti/covidbert-nli").cuda()

In [7]:
def isNaN(num):
    return num != num

In [8]:
torch.cuda.empty_cache()

In [ ]:
path = "C:/Users/User/Documents/NTNU/NLP/CORD-19/Embeddings/786/"
#n_models = 3
#n_fields = 2
dim = 786

start = time.time()
m = start

field_name = 'abstract'

for idx, row in docs.iterrows():
    embedding = {}
    if (idx < 10):
        torch.cuda.empty_cache()
        cord_uid = row["cord_uid"]
        embedding = {}
        file = path + cord_uid + ".txt"
        with open(file, "rb") as fp:   # Unpickling
            embedding = pickle.load(fp)

        #models = [bert_model, scibert_model, covid_model]
        model_name = ['bert-base-uncased', "allenai/scibert_scivocab_uncased", "gsarti/covidbert-nli"]
        #tokenizers = [bert_tokenizer ,scibert_tokenizer, covid_tokenizer]
        #fields = ["title", "abstract"]
        
        #embedding["models"][model_name[i]] = {}
        field = row[field_name]

        res = isinstance(field, str) 
        if (not res) or isNaN(field):
            field = row["title"]


        embedd = tokenize(bert_model, bert_tokenizer, field)
        embedding["models"][model_name[model_id]][field_name] = embedd
   

        ## Timing the loops
        if (idx % 1000) == 0:
            print(idx)
            print("Time (1000 iterations):", round(time.time() - m,1))
            m = time.time()

        with open(file, "wb") as fp:   #Pickling
            pickle.dump(embedding, fp)

In [ ]:
string = "Until now we have talked about term matching methods for information retrieval. While these methods are good they do have some issues. One issue is when the user is unfamiliar with what terms are used in relevant documents. A user might search for what percentage of people infected by covid die?, while the more 'correct' way might be What is the Covid case fatality rate. These two sentences have more or less the same semantic meaning, but can lead to different results when using term matching. What if we could try to extract the semantic meaning of a sentence, and match it with the query? For this task we will introduce sentence embedding models."
string2 = "Before looking more into sentence embedding models we will look at a simpler task; word embedding. The point of this task is to map a word into a vector. This vector should represents the meaning of the word. When the meaning of a word is represented as a vector it can be easily interpreted by a computer. One can understand the main idea of word embedding by a simple equation "
string3 = "During the global covid-19 pandemic it has been important for researchers, politicians and others to be able to access the newest and most relevant research relating to COVID-19. The Semantic Scholar team at the Allen Institute for AI has partnered with leading research groups and released a dataset called CORD-19 \cite{Wang2020CORD19TC}. The dataset aims to keep a corpus of COVID-19 research articles so researchers can apply recent advances in natural language processing to aid in the global effort against the pandemic. The CORD-19 dataset contains research articles relating to COVID-19. This includes articles on similar viruses like SARS and MERS, pandemics and other relevant topics. The dataset is updated daily. "
tokenize(covid_model, covid_tokenizer, string+string2+string3+string+string2)


In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
scibert_tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
scibert_model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

In [ ]:
covid_tokenizer = AutoTokenizer.from_pretrained("gsarti/covidbert-nli")
covid_model = AutoModel.from_pretrained("gsarti/covidbert-nli")

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\"
docs = pd.read_csv(path + "crod_19_only_rel.csv")
docs.shape
docs.head()

In [ ]:
docs.head(2)

## Bert embeddings

### Title

In [ ]:
path = "C:/Users/User/Documents/NTNU/NLP/CORD-19/Embeddings/BERT/786/title/"
for idx, row in docs.iterrows():
    BERT_embeddings_title = {}
    sha = row["sha"]
    cord_uid = row["cord_uid"]
    BERT_embeddings_title[row["sha"]] = tokenize(bert_model, bert_tokenizer, row["title"])
    if (idx % 10000) == 0:
        print(idx)
    file = path + cord_uid + ".txt"
    with open(file, "wb") as fp:   #Pickling
        pickle.dump(BERT_embeddings_title, fp)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "BERT_embedding_title.txt", "wb") as fp:   #Pickling
    pickle.dump(BERT_embeddings_title, fp)
    

### Abstract

In [ ]:
BERT_embeddings_abstract = {}
for idx, row in docs.iterrows():
    BERT_embeddings_abstract[row["sha"]] = tokenize(bert_model, bert_tokenizer, row["abstract"])
    if (idx % 10000) == 0:
        print(idx)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "BERT_embedding_abstract.txt", "wb") as fp:   #Pickling
    pickle.dump(BERT_embeddings_abstract, fp)

## SciBert embeddings

### Title

In [ ]:
SciBERT_embeddings_title = {}
for idx, row in docs.iterrows():
    SciBERT_embeddings_title[row["sha"]] = tokenize(scibert_model, scibert_tokenizer, row["title"])
    if (idx % 10000) == 0:
        print(idx)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "SciBERT_embedding_title.txt", "wb") as fp:   #Pickling
    pickle.dump(SciBERT_embeddings_title, fp)

### Abstract

In [ ]:
SciBERT_embeddings_abstract = {}
for idx, row in docs.iterrows():
    SciBERT_embeddings_abstract[row["sha"]] = tokenize(scibert_model, scibert_tokenizer, row["abstract"])
    if (idx % 10000) == 0:
        print(idx)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "SciBERT_embedding_abstract.txt", "wb") as fp:   #Pickling
    pickle.dump(SciBERT_embeddings_abstract, fp)

## CordBert embeddings

### Title

In [ ]:
CordBERT_embeddings_title = {}
for idx, row in docs.iterrows():
    CordBERT_embeddings_title[row["sha"]] = tokenize(covid_model, covid_tokenizer, row["title"])
    if (idx % 10000) == 0:
        print(idx)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "CordBERT_embedding_title.txt", "wb") as fp:   #Pickling
    pickle.dump(CordBERT_embeddings_title, fp)

### Abstract

In [ ]:
CordBERT_embeddings_abstract = {}
for idx, row in docs.iterrows():
    CordBERT_embeddings_abstract[row["sha"]] = tokenize(covid_model, covid_tokenizer, row["abstract"])
    if (idx % 10000) == 0:
        print(idx)

In [ ]:
path = "C:\\Users\\User\\OneDrive - NTNU\\NTNU\\Prosjekt oppgave NLP\\dataset\\CORD-19\\embeddings\\"

with open(path + "CordBERT_embedding_abstract.txt", "wb") as fp:   #Pickling
    pickle.dump(CordBERT_embeddings_abstract, fp)

## Embedding dim=786

In [ ]:
def isNaN(num):
    return num != num

In [ ]:
path = "C:/Users/User/Documents/NTNU/NLP/CORD-19/Embeddings/786/"
n_models = 3
n_fields = 2
dim = 786

start = time.time()
m = start

for idx, row in docs.iterrows():
    embedding = {}
    if (idx>21999):
        sha = row["sha"]
        cord_uid = row["cord_uid"]
        embedding["sha"] = sha
        embedding["cord_uid"] = cord_uid
        embedding["dim"] = dim
        embedding["models"] = {}

        models = [bert_model, scibert_model, covid_model]
        model_name = ['bert-base-uncased', "allenai/scibert_scivocab_uncased", "gsarti/covidbert-nli"]
        tokenizers = [bert_tokenizer ,scibert_tokenizer, covid_tokenizer]
        fields = ["title", "abstract"]

        for i in range(n_models):
            embedding["models"][model_name[i]] = {}
            for j in range(n_fields):
                field = row[fields[j]]

                res = isinstance(fields[j], str) 
                if (not res) or isNaN(field):
                    field = row["title"]
              
                
                embedd = tokenize(models[i], tokenizers[i], field)
                embedding["models"][model_name[i]][fields[j]] = embedd
                torch.cuda.empty_cache()

        ## Timing the loops
        if (idx % 1000) == 0:
            print(idx)
            print("Time (1000 iterations):", round(time.time() - m,1))
            m = time.time()

        file = path + cord_uid + ".txt"
        with open(file, "wb") as fp:   #Pickling
            pickle.dump(embedding, fp)

In [ ]:
"""
{
sha: sha,
cord_uid: cord_uid,
dim: dim
models: {model_name1: {title: [...],
                       abstract: [...]},
         model_name2: {title: [...],
                       abstract: [...]},
        ...                   
}
}


"""

In [ ]:
torch.cuda.empty_cache()